<a href="https://colab.research.google.com/github/natrask/ENM5310/blob/main/Lecture0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We'll begin by giving a tutorial on tensorflow for performing regression tasks. I choose to work in Tensorflow 1 - this provides the most transparent view of what the model is actually doing. These days most prefer pytorch/jax, or at least tensorflow 2. You can submit homeworks in whichever of these you prefer to work in, but may not use external libraries (e.g. only use Keras when given explicit permission in the assignment, no importing of libraries with pre-implemented architectures, etc). This will let us understand architectures at a low level and avoid "code-glue" where we stick random bits of code together without understanding why/how they work.